In [0]:
import os

os.chdir("/content/drive/My Drive/Colab Notebooks/Code_search_engine")

In [0]:
!pip install transformers
!pip install tensorboardX

In [0]:
import torch

import transformers
from transformers import (WEIGHTS_NAME,BertConfig, BertForMaskedLM, BertTokenizer)

In [0]:
import json

with open('/content/data/test_original_function.json') as json_file:
    function_data = json.load(json_file)

In [0]:
import pandas as pd
import numpy as np

df_docstring = pd.read_csv("./data/test.docstring",sep='\t', names=["docstring"])[:20000]
df_lineage = pd.read_csv("./data/test.lineage",sep='\t', names=["Repo"])[:20000]

In [0]:
docstrings_avg_vec = np.load('./docstrings_avg_vecs.npy',allow_pickle=True)

In [0]:
model_path = "./output/pytorch_model.bin"

config = BertConfig.from_json_file('./output/config.json')
config.output_hidden_states = True


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForMaskedLM.from_pretrained("bert-base-uncased",config=config)
model.load_state_dict(torch.load(model_path))
model.eval()

# Creating the search engine:

In [0]:
!pip install nmslib

In [0]:
import nmslib
import numpy

# create a random matrix to index
data = docstrings_avg_vec

# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

In [59]:
user_input = "create a NN network."

input_ids = torch.tensor(tokenizer.encode(user_input, add_special_tokens=True)).unsqueeze(0)
input_ids

tensor([[ 101, 3443, 1037, 1050, 2078, 2897, 1012,  102]])

In [60]:
outputs = model(input_ids, masked_lm_labels=input_ids)

embeddings = outputs[2][-1]
print(embeddings.shape)
embeddings = embeddings.detach().numpy()[0]

size = embeddings.shape[0]
sum_array = [sum(x) for x in zip(*embeddings)]
avg_array = [sum_array[i]/size for i in range(len(sum_array))]

torch.Size([1, 8, 768])


In [0]:
# query for the nearest neighbours of the first datapoint
ids, distances = index.knnQuery(avg_array, k=10)

In [63]:
best_result_id = ids[1]

function_data[best_result_id]

'@privileged.default.entrypoint\ndef create_netns(name, **kwargs):\n    """Create a network namespace.\n\n    :param name: The name of the namespace to create\n    """\n    try:\n        netns.create(name, **kwargs)\n    except OSError as e:\n        if e.errno != errno.EEXIST:\n            raise\n'